In [1]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor

##### Dependencies
!pip install transformers
!pip3 install torch torchvision torchaudio
!pip install sentencepiece

In [1]:
!pip install transformers
!pip3 install torch torchvision torchaudio
!pip install sentencepiece

In [8]:
if torch.cuda.is_available():
   dev = torch.device("cuda:0")
   print("Running on the GPU")
else:
   dev = torch.device("cpu")
   print("Running on the CPU")

Running on the CPU


In [9]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base',
                                             return_dict=True)
optimizer = Adafactor(model.parameters(),lr=1e-3,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)
model.to(dev)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [ ]:
train_df=pd.read_csv('dataframe_slice.csv', error_bad_lines=False,engine='python',index_col=0)

train_df=train_df.iloc[  :10000,:]
train_df=train_df.sample(frac = 1)

batch_size=8
num_of_batches=len(train_df)//batch_size

In [4]:
train_df

,title,description,title_len,desc_len
6001,Command Communications Comswitch 5500 3-Port P...,['Command Communications Comswitch 5500 3-Port...,80,1217
4524,Agfa ePhoto CL30 Clik! 1MP Digital Camera,"[""Agfa's ePhoto CL30 Clik! digital camera feat...",41,990
14193,Magnavox MANT-075 VHF/UHF/FM Behind the Set In...,['The Philips MANT075 is an Indoor Passive Ant...,89,270
18706,Nikon C-PL1L Slip-in 52mm Circular Polarizing ...,['The Nikon 52mm C-PL1L Drop-in Circular Polar...,52,636
25847,Polk Audio Atrium 65SDI All-Weather Stereo Spe...,['POLK Atrium 65 - This unique speaker might w...,66,2758
...,...,...,...,...
1543,PROFESSIONAL 10 ft Asus Transformer Pad Infini...,['PROFESSIONAL 10 ft Samsung Droid Smartphone ...,137,108
24026,"Viewsonic G70F-2 17"" CRT Monitor",['VIEWSONIC (G70F-2) ViewSonic G70f - Display ...,32,103
28018,Cool Lift Portable Notebook Cooling Pad,['The cool lift notepad cooling pad is a great...,39,606
13748,Network Everywhere NC100 10/100 Network Card,"[""Manufacturer's Part # NC100<BR>This product ...",44,1067


In [5]:
from IPython.display import HTML, display
def progress(loss,value, max=100):
    return HTML(""" Batch loss :{loss}
      <progress    
value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

In [14]:
model.train()
num_of_epochs=1
loss_per_10_steps=[]
for epoch in range(1,num_of_epochs+1):
    print('Running epoch: {}'.format(epoch))
  
    running_loss=0

    out = display(progress(1, num_of_batches+1), display_id=True)
    for i in range(num_of_batches):
        inputbatch=[]
        labelbatch=[]
        new_df=train_df[i*batch_size:i*batch_size+batch_size]
        for indx,row in new_df.iterrows():
            inputs = row['title']
            labels = row['description']
            inputbatch.append(inputs)
            labelbatch.append(labels)
        inputbatch=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=100,return_tensors='pt')["input_ids"]
        labelbatch=tokenizer.batch_encode_plus(labelbatch,padding=True,max_length=100,return_tensors="pt") ["input_ids"]
        inputbatch=inputbatch.to(dev)
        labelbatch=labelbatch.to(dev)

    # clear out the gradients of all Variables 
        optimizer.zero_grad()

    # Forward propogation
        outputs = model(input_ids=inputbatch, labels=labelbatch)
        loss = outputs.loss
        loss_num=loss.item()
        logits = outputs.logits
        running_loss+=loss_num
        if i%10 ==0:      
            loss_per_10_steps.append(loss_num)
        out.update(progress(loss_num,i, num_of_batches+1))

    # calculating the gradients
        loss.backward()

    #updating the params
        optimizer.step()
    
    running_loss=running_loss/int(num_of_batches)
    print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))

Running epoch: 1


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\chunshen\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-746962faaab4>", line 28, in <module>
    outputs = model(input_ids=inputbatch, labels=labelbatch)
  File "C:\Users\chunshen\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "C:\Users\chunshen\anaconda3\lib\site-packages\transformers\models\t5\modeling_t5.py", line 1633, in forward
    lm_logits = self.lm_head(sequence_output)
  File "C:\Users\chunshen\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "C:\Users\chunshen\anaconda3\lib\site-packages\torch\nn\modules\linear.py", line 96, in forward
    return F.linear(input, self.weight, self.bias)
  File "C:\Users\chunshen\anaconda3\lib\site-packages\torch

TypeError: object of type 'NoneType' has no len()

In [ ]:
tokenizer.